**<h1><center>Computational Models for Decision Making<center><h1>**

In [1]:
%%capture --no-display
%pylab inline
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

import numba
import cupy as cp
from numba import cuda, prange
from numba.cuda import random
from numba import jit, njit, vectorize
from numba.core.errors import NumbaPerformanceWarning
import warnings
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)
import GPUtil


In [2]:
# Functions
def ReLU(input):
    '''Takes an input value and performs rectified linear unit operation i.e., 
    A = A   if A >= 0
    A = 0   if A <  0
    '''
    return input * (input >= 0)

def get_unsigned_coherence_matrix(normalized_signed_stimulus, num_choices = 2):
    '''
    Most of the decision-making studies use 2AFC task where usually choices are signed as positive for one choice and negative for another choice.
    Through this function we seperate this nomenclature by oupting two unsigned inputs rather than one signed input.
    
    Args: 
        normalized_signed_stimulus: signed stimulus difficulty e.g., coherence with num_trials x stop_time dimensions
    
    return:
        normalized_unsigned_stimulus: For 2-AFC task, returns stimulus matrix with dimensions num_trials x num_choices x stop_time
    '''
    
    # Input validation
    if not isinstance(normalized_signed_stimulus, numpy.ndarray):
        raise TypeError('Input must be a 1D or 2D numpy array with num_trials x stop_time dimensions')
        
    if normalized_signed_stimulus.ndim > 2:
        raise TypeError('Input must be a 1D or 2D numpy array with num_trials x stop_time dimensions')
                
    if normalized_signed_stimulus.ndim == 1:
        if len(normalized_signed_stimulus)<2:
            raise TypeError('Input must be a 1D or 2D numpy array with num_trials x stop_time dimensions with minumum 2 time points')
        else:
            normalized_unsigned_stimulus = np.zeros((num_choices, normalized_signed_stimulus.shape[0]), dtype=float32)
            normalized_unsigned_stimulus[0,:] = normalized_signed_stimulus * (normalized_signed_stimulus>0)   # If stimulus was positive keep it same for first cell else zero
            normalized_unsigned_stimulus[1,:] = -normalized_signed_stimulus * (normalized_signed_stimulus<0)   # If stimulus was negative take it's negative for second cell else zero
    
    if normalized_signed_stimulus.ndim == 2:
        normalized_unsigned_stimulus = np.zeros((normalized_signed_stimulus.shape[0], num_choices, normalized_signed_stimulus.shape[1]), dtype=float32)
        normalized_unsigned_stimulus[:,0,:] = normalized_signed_stimulus * (normalized_signed_stimulus>0)   # If stimulus was positive keep it same for first cell else zero
        normalized_unsigned_stimulus[:,1,:] = -normalized_signed_stimulus * (normalized_signed_stimulus<0)   # If stimulus was negative take it's negative for second cell else zero
 

    return normalized_unsigned_stimulus


# @cuda.jit
# def ReLU_cuda(input):
#     return input * (input > 0)

def get_time_dependent_bound(initial_bound, rate, delay, stop_time=10000):
    '''
    Generates time-dependent bound such as collapsing bound array
    args:
        initial_bound: Initial height of bounds
        rate: Rate of exponential decay. Negative means decreasing bound and +ve means increasings bounds
        delay: What point should decay start
        stop_time: To determine length of array
        
    returns:
        time_dependent_bound: 1D array with either exponential decay or rise
    '''
    
    time_dependent_bound = cp.ones(stop_time)*initial_bound
    
    return time_dependent_bound.astype(float32)
    
def get_time_dependent_variability(initial_variability, time_coefficient=0, stop_time=10000):
    '''
    Generates linear time-dependent sigma either increasing, decreasing or constant
    args:
        initial_variability: Initial value of diffusion variability (sigma)
        time_coefficient: Rate of linear time-dependency. Negative means decreasing bound and +ve means increasings diffusion variability. Default value 0 i.e., constant variability
        stop_time: To determine length of array (Default value 10000 or 10 seconds)
        
    returns:
        time_dependent_variability: 1D array with either constant or linearly time-dependent diffusion variability
    '''
    
    time_dependent_variability = (cp.ones(stop_time)*initial_variability) + (time_coefficient*cp.arange(stop_time))
    
    return time_dependent_variability.astype(float32)


In [245]:
# cp.random.seed()

# For Dynamic Coherence
@cuda.jit
def DDM_kernel_for_delay(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time, rng_states): 
    tr = cuda.grid(1)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))   
    if tr_in_bounds:
        decision_variable = starting_point                
        for t in range(stimulus.shape[2]):
            diffusion_step = ((stimulus[tr,0,t]-stimulus[tr,1,t]) * drift_gain)+ (random.xoroshiro128p_normal_float32(rng_states, tr)*drift_variability[t])   
            decision_variable += diffusion_step + drift_offset      # update decision variable
            if decision_variable > decision_bound[t] or decision_variable < -decision_bound[t]:       # checking if decision bound is reached
                decision[tr] =  2*(decision_variable>0) - 1    # np.sign(dv) alternative
                reaction_time[tr] = t+nondecision_time
                break  
                
#for vectors arranged column-wise
@cuda.jit
def mat_sum_cuda(vec, output):
    output[0] = 0
    for _, val in enumerate(vec.flat):
        output[0] += val
        
@cuda.jit
def vec_multiplication_cuda(vector, scaler, output):
    ouput = 0
    for i in range(len(vector)):
        output[i] = vector[i]*scaler
        

# For Dynamic Coherence
@cuda.jit
def independent_accumulating_kernel(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states): 
    
    tr = cuda.grid(1)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))           
    
    decision_variable = cuda.local.array(num_choices, dtype=numba.float32)
    sum_decision_variables = cuda.local.array(1, dtype=numba.float32)
    drift_rate = cuda.local.array(num_choices, dtype=numba.float32)
#     ddm_like_dr_inhibition_step = cuda.local.array(0, dtype=numba.float32)
    temp = cuda.local.array(1, dtype=numba.float32)

    
    if tr_in_bounds: 
        decision_variable = starting_point
        mat_sum_cuda(decision_variable, sum_decision_variables)
        
            
        for t in range(stimulus.shape[2]):
            mat_sum_cuda(decision_variable, sum_decision_variables)
            vec_multiplication_cuda(stimulus[tr,:,t], drift_gain, drift_rate)
            
            for accumulator in range(stimulus.shape[1]):   
                
                diffusion_step = drift_rate[accumulator] + drift_offset[accumulator]#  + (random.xoroshiro128p_normal_float32(rng_states, tr)*drift_variability[t])            
                leak_step = leak * decision_variable[accumulator]
                lateral_dv_inhibition_step = lateral_inhibition * (sum_decision_variables[0] - decision_variable[accumulator])      
                ddm_like_dr_inhibition_step = numba.float32(0.0) 
                mat_sum_cuda(drift_rate,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_rate[accumulator]   # Collecting all drift-rate except current
                mat_sum_cuda(drift_offset,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_offset[accumulator] # Collecting all drift-offset except current
                
                decision_step = decision_variable[accumulator] \
                                + diffusion_step \
                                - leak_step \
                                - lateral_dv_inhibition_step* \
                                - ddm_like_dr_inhibition_step*neural_ddm
#                                 neural_ddm*(cp.sum(drift_rate + drift_offset) - drift_rate[accumulator] - drift_offset[accumulator])      # update decision variable
                
                decision_variable[accumulator] = decision_step * (decision_step > 0)      # Implementing ReLU operation i.e., only non-negative value else zero
        
                if decision_variable[accumulator] > decision_bound[t]:
                    decision[tr] = accumulator
                    reaction_time[tr] = t + nondecision_time
                    break

#             if (decision_variable > decision_bound[t]).any():
#                 decision[tr] =  decision_variable.argmax(0)
#                 reaction_time[tr] = t + nondecision_time
#                 break
                
      
        
        

num_trials = 50#000
stop_time = 100#00
num_choices = 2
coh = 100
coherence = np.ones((num_trials, stop_time), dtype=float32)*coh

starting_point = cp.array(np.zeros(num_choices), dtype=float32)             
drift_offset = cp.array(np.zeros(num_choices), dtype=float32)
drift_gain = 10e-6             # drift gain
drift_variability = 10e-3      # diffusion variability
nondecision_time = 100         # Non-decision time (msec)
decision_bound = 1
bound_rate = 0
bound_delay = 0
lateral_inhibition = 0
leak = 0
neural_ddm = 0

# Dynamic time-dependent variables
stimulus_np = get_unsigned_coherence_matrix(coherence)
decision_bound_cp = get_time_dependent_bound(decision_bound, bound_rate, bound_delay)
drift_variability_cp = get_time_dependent_variability(drift_variability)
stimulus_cp = cp.asarray(stimulus_np)

decision = cp.empty(stimulus_cp.shape[0])*cp.NaN
reaction_time = cp.empty(stimulus_cp.shape[0])*cp.NaN
blockdim = (256)
griddim = stimulus_cp.shape[0] // (blockdim) + 1
rng_states = random.create_xoroshiro128p_states(griddim * blockdim, seed=1)
cuda.synchronize()
%time independent_accumulating_kernel[griddim, blockdim](stimulus_cp, starting_point, drift_gain, drift_variability_cp, drift_offset, decision_bound_cp, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states) 
cuda.synchronize()


Wall time: 360 ms


### With 2nd level parallelization

In [286]:
# cp.random.seed()

# For Dynamic Coherence
@cuda.jit
def DDM_kernel_for_delay(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time, rng_states): 
    tr = cuda.grid(1)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))   
    if tr_in_bounds:
        decision_variable = starting_point                
        for t in range(stimulus.shape[2]):
            diffusion_step = ((stimulus[tr,0,t]-stimulus[tr,1,t]) * drift_gain)# + (random.xoroshiro128p_normal_float32(rng_states, tr)*drift_variability[t])   
            decision_variable += diffusion_step + drift_offset      # update decision variable
            if decision_variable > decision_bound[t] or decision_variable < -decision_bound[t]:       # checking if decision bound is reached
                decision[tr] =  2*(decision_variable>0) - 1    # np.sign(dv) alternative
                reaction_time[tr] = t+nondecision_time
                break  
    
                
#for vectors arranged column-wise
@cuda.jit
def mat_sum_cuda(vec, output):
    output[0] = 0
    for _, val in enumerate(vec.flat):
        output[0] += val
        
@cuda.jit
def vec_multiplication_cuda(vector, scaler, output):
    ouput = 0
    for i in range(len(vector)):
        output[i] = vector[i]*scaler
        

# For Dynamic Coherence
@cuda.jit
def LCA_kernel_for_delay(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states): 
    
    tr, accumulator = cuda.grid(2)
    stridex,stridey = cuda.gridsize(2)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))        
    accumulator_in_bounds = (accumulator >= 0) and (accumulator <= (num_choices - 1))          
    
    decision_variable = cuda.local.array(num_choices, dtype=numba.float32)
    sum_decision_variables = cuda.local.array(1, dtype=numba.float32)
    drift_rate = cuda.local.array(num_choices, dtype=numba.float32)
    temp = cuda.local.array(1, dtype=numba.float32)

    
    if tr_in_bounds: 
        decision_variable = starting_point
        mat_sum_cuda(decision_variable, sum_decision_variables)
        
            
        for t in range(stimulus.shape[2]):
            mat_sum_cuda(decision_variable, sum_decision_variables)
            vec_multiplication_cuda(stimulus[tr,:,t], drift_gain, drift_rate)
            
            if accumulator_in_bounds:
                
                idx = tr * stridey + accumulator
#                 random.xoroshiro128p_normal_float32(rng_states, idx)*drift_variability[0]
                diffusion_step = drift_rate[accumulator] + drift_offset[accumulator]#  + (random.xoroshiro128p_normal_float32(rng_states, idx)*drift_variability[t])            
                leak_step = leak * decision_variable[accumulator]
                lateral_dv_inhibition_step = lateral_inhibition * (sum_decision_variables[0] - decision_variable[accumulator])      
                ddm_like_dr_inhibition_step = numba.float32(0.0) 
                mat_sum_cuda(drift_rate,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_rate[accumulator]   # Collecting all drift-rate except current
                mat_sum_cuda(drift_offset,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_offset[accumulator] # Collecting all drift-offset except current
                
                decision_step = decision_variable[accumulator] \
                                + diffusion_step \
                                - leak_step \
                                - lateral_dv_inhibition_step* \
                                - ddm_like_dr_inhibition_step*neural_ddm
                
                decision_variable[accumulator] = decision_step * (decision_step > 0)      # Implementing ReLU operation i.e., only non-negative value else zero
        
                if decision_variable[accumulator] > decision_bound[t]:
                    decision[tr] = accumulator
                    reaction_time[tr] = t + nondecision_time
                    break
            else:
                continue
            break

      
        
        

num_trials = 50#000
stop_time = 10000
num_choices = 2
coh = 100
coherence = np.ones((num_trials, stop_time), dtype=float32)*coh

starting_point = cp.array(np.zeros(num_choices), dtype=float32)             
drift_offset = cp.array(np.zeros(num_choices), dtype=float32)
drift_gain = float32(10e-6)             # drift gain
drift_variability = float32(10e-3)      # diffusion variability
nondecision_time = float32(100)         # Non-decision time (msec)
decision_bound = 1
bound_rate = 0
bound_delay = 0
lateral_inhibition = 0
leak = 0
neural_ddm = 0

# Dynamic time-dependent variables
stimulus_np = get_unsigned_coherence_matrix(coherence)
decision_bound_cp = get_time_dependent_bound(decision_bound, bound_rate, bound_delay)
drift_variability_cp = get_time_dependent_variability(drift_variability)
stimulus_cp = cp.asarray(stimulus_np)

decision = cp.empty(stimulus_cp.shape[0])*cp.NaN
reaction_time = cp.empty(stimulus_cp.shape[0])*cp.NaN
blockdim = (128,8)
griddim = ((stimulus_cp.shape[0] // blockdim[0]) + 1, (num_choices // blockdim[1]) + 1)
rng_states = random.create_xoroshiro128p_states(int(np.prod(blockdim) * np.prod(griddim)), seed=1)
cuda.synchronize()
%time LCA_kernel_for_delay[griddim, blockdim](stimulus_cp, starting_point, drift_gain, drift_variability_cp, drift_offset, decision_bound_cp, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states) 
cuda.synchronize()


Wall time: 364 ms



<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Testing 

## DDM (finalized)

In [ ]:
# DDM 
# For Dynamic Coherence
def DDM_kernel(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time): 
    for tr in range(stimulus.shape[0]):
        decision_variable = 0.0              
        for t in range(stimulus.shape[2]):
            diffusion_step = ((stimulus[tr,0,t]-stimulus[tr,1,t]) * drift_gain)
            decision_variable += diffusion_step + drift_offset      # update decision variable
            if decision_variable > decision_bound[t] or decision_variable < -decision_bound[t]:       # checking if decision bound is reached
                decision[tr] =  2*(decision_variable>0) - 1    # np.sign(dv) alternative
                reaction_time[tr] = t+nondecision_time
                break  

def DDM_function(coherence, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time):
    for tr in range(coherence.shape[0]):
        decision_variable = 0
        for t in range(coherence.shape[-1]):
            decision_variable = decision_variable + (coherence[tr, t]*drift_gain) + drift_offset
                        
            if decision_variable > decision_bound[t] or decision_variable < -decision_bound[t]:       # checking if decision bound is reached
                decision[tr] =  2*(decision_variable>0) - 1    # np.sign(dv) alternative
                reaction_time[tr] = t+nondecision_time
                break   
                

@cuda.jit
def DDM_kernel_for_delay(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time, rng_states): 
    tr = cuda.grid(1)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))   
    if tr_in_bounds:
        decision_variable = starting_point                
        for t in range(stimulus.shape[2]):
            diffusion_step = ((stimulus[tr,0,t]-stimulus[tr,1,t]) * drift_gain)# + (random.xoroshiro128p_normal_float32(rng_states, tr)*drift_variability[t])   
            decision_variable += diffusion_step + drift_offset      # update decision variable
            if decision_variable > decision_bound[t] or decision_variable < -decision_bound[t]:       # checking if decision bound is reached
                decision[tr] =  2*(decision_variable>0) - 1    # np.sign(dv) alternative
                reaction_time[tr] = t+nondecision_time
                break  
    
    
    
    
    
    
num_choices = 2
coherence = np.ones((2,2000))*36     # 100
coherence[0,200:600] = -100
starting_point = 0 #np.array(np.zeros(1), dtype=float32)             
drift_offset = 0 #np.array(np.zeros(1), dtype=float32)
drift_gain = float32(5e-5)             # drift gain
drift_variability = float32(10e-3)      # diffusion variability
nondecision_time = float32(100)         # Non-decision time (msec)
decision_bound = 1
bound_rate = 0
bound_delay = 0
lateral_inhibition = 0
leak = 0
neural_ddm = 0
# Dynamic time-dependent variables
stimulus = get_unsigned_coherence_matrix(coherence)
decision_bound = cp.asnumpy(get_time_dependent_bound(decision_bound, bound_rate, bound_delay))
drift_variability = cp.asnumpy(get_time_dependent_variability(drift_variability))
decision1 = np.empty(stimulus.shape[0])*np.NaN
reaction_time1 = np.empty(stimulus.shape[0])*np.NaN
decision2 = np.empty(stimulus.shape[0])*np.NaN
reaction_time2 = np.empty(stimulus.shape[0])*np.NaN

stimulus_cp= cp.asarray(stimulus)
decision_bound_cp = cp.asarray(decision_bound)
drift_variability_cp = cp.asarray(drift_variability)
decision3 = cp.empty(stimulus.shape[0])*cp.NaN
reaction_time3 = cp.empty(stimulus.shape[0])*cp.NaN
blockdim = (256)
griddim = stimulus_cp.shape[0] // (blockdim) + 1
rng_states = random.create_xoroshiro128p_states(griddim * blockdim, seed=1)


DDM_kernel(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision1, reaction_time1)
DDM_function(coherence, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision2, reaction_time2)
cuda.synchronize()
DDM_kernel_for_delay[griddim, blockdim](stimulus_cp, starting_point, drift_gain, drift_variability_cp, drift_offset, decision_bound_cp, nondecision_time, decision3, reaction_time3, rng_states)
cuda.synchronize()    
    
print(decision1, decision2, cp.asnumpy(decision3))
print(reaction_time1, reaction_time2, cp.asnumpy(reaction_time3))

[-1.  1.] [-1.  1.] [-1.  1.]
[572. 655.] [572. 655.] [572. 655.]



<br><br><br><br>

## Independent Accumulator Models

In [316]:

def independent_accumulator(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, decision, reaction_time): 
    for tr in range(stimulus.shape[0]):
        decision_variable = starting_point.copy()      
        sum_decision_variable = decision_variable.sum()
        
        for t in range(stimulus.shape[2]):
            sum_decision_variable = decision_variable.sum()
            drift_rate = stimulus[tr,:,t] * drift_gain
            for accumulator in range(stimulus.shape[1]):
                
                diffusion_step = drift_rate[accumulator] + drift_offset[accumulator]
                leak_step = leak * decision_variable[accumulator]
                lateral_dv_inhibition_step = lateral_inhibition * (sum_decision_variable - decision_variable[accumulator])  # Lateral inhibition from all decision_variables except self
                ddm_like_dr_inhibition_step = drift_rate.sum() - drift_rate[accumulator] + drift_offset.sum() - drift_offset[accumulator]
                
                decision_step = decision_variable[accumulator] \
                                + diffusion_step \
                                - leak_step \
                                - lateral_dv_inhibition_step \
                                - ddm_like_dr_inhibition_step*neural_ddm
                
                decision_variable[accumulator] = decision_step * (decision_step > 0)
                if decision_variable[accumulator] > decision_bound[t]:
                    decision[tr] = accumulator
                    reaction_time[tr] = t + nondecision_time
                    break
            else:
                continue
            break

                
                
num_choices = 2
coherence = np.ones((2,2000))*100     # 100
coherence[0,10:100] = -100
coherence[1,10:150] = -500
starting_point = np.zeros(num_choices, dtype=float32)             
drift_offset = np.zeros(num_choices, dtype=float32)
drift_gain = float32(10e-5)             # drift gain
drift_variability = float32(10e-3)      # diffusion variability
nondecision_time = float32(100)         # Non-decision time (msec)
decision_bound = 1
bound_rate = 0
bound_delay = 0
lateral_inhibition = 0
leak = 0
neural_ddm = 0
# Dynamic time-dependent variables
stimulus_cp = cp.asarray(get_unsigned_coherence_matrix(coherence))
decision_bound_cp = get_time_dependent_bound(decision_bound, bound_rate, bound_delay)
drift_variability_cp = get_time_dependent_variability(drift_variability)
stimulus_np = cp.asnumpy(stimulus_cp)
decision_bound_np = cp.asnumpy(decision_bound_cp)
drift_variability_np = cp.asnumpy(drift_variability_cp)


decision1 = np.empty(stimulus_np.shape[0])*np.NaN
reaction_time1 = np.empty(stimulus_np.shape[0])*np.NaN
decision2 = cp.empty(stimulus_cp.shape[0])*cp.NaN
reaction_time2 = cp.empty(stimulus_cp.shape[0])*cp.NaN
decision3 = cp.empty(stimulus_cp.shape[0])*cp.NaN
reaction_time3 = cp.empty(stimulus_cp.shape[0])*cp.NaN
                
independent_accumulator(stimulus_np, starting_point, drift_gain, drift_variability_np, drift_offset, decision_bound_np, nondecision_time, decision1, reaction_time1)           
                
blockdim = (256)
griddim = stimulus_cp.shape[0] // (blockdim) + 1
rng_states = random.create_xoroshiro128p_states(griddim * blockdim, seed=1)
cuda.synchronize()
independent_accumulating_kernel[griddim, blockdim](stimulus_cp, starting_point, drift_gain, drift_variability_cp, drift_offset, decision_bound_cp, nondecision_time, lateral_inhibition, leak, neural_ddm, decision2, reaction_time2, rng_states) 
cuda.synchronize()

print(decision1, cp.asnumpy(decision2))#, cp.asnumpy(decision3))
print(reaction_time1, cp.asnumpy(reaction_time2))#, cp.asnumpy(reaction_time3))

[0. 1.] [1. 1.]
[290. 129.] [130. 129.]


In [314]:
                
#for vectors arranged column-wise
@cuda.jit
def mat_sum_cuda(vec, output):
    output[0] = 0
    for _, val in enumerate(vec.flat):
        output[0] += val
        
@cuda.jit
def vec_multiplication_cuda(vector, scaler, output):
    ouput = 0
    for i in range(len(vector)):
        output[i] = vector[i]*scaler
        
                
# For Dynamic Coherence
@cuda.jit
def independent_accumulating_kernel(stimulus, starting_point, drift_gain, drift_variability, drift_offset, decision_bound, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states): 
    
    tr = cuda.grid(1)
    tr_in_bounds = (tr >= 0) and (tr <= (stimulus.shape[0] - 1))           
    
    decision_variable = cuda.local.array(num_choices, dtype=numba.float32)
    sum_decision_variables = cuda.local.array(1, dtype=numba.float32)
    drift_rate = cuda.local.array(num_choices, dtype=numba.float32)
#     ddm_like_dr_inhibition_step = cuda.local.array(0, dtype=numba.float32)
    temp = cuda.local.array(1, dtype=numba.float32)

    
    if tr_in_bounds: 
        decision_variable = starting_point
        mat_sum_cuda(decision_variable, sum_decision_variables)
        
        for t in range(stimulus.shape[2]):
            mat_sum_cuda(decision_variable, sum_decision_variables)
            vec_multiplication_cuda(stimulus[tr,:,t], drift_gain, drift_rate)
            
            for accumulator in range(stimulus.shape[1]):   
                
                diffusion_step = drift_rate[accumulator] + drift_offset[accumulator]#  + (random.xoroshiro128p_normal_float32(rng_states, tr)*drift_variability[t])            
                leak_step = leak * decision_variable[accumulator]
                lateral_dv_inhibition_step = lateral_inhibition * (sum_decision_variables[0] - decision_variable[accumulator])      
                ddm_like_dr_inhibition_step = numba.float32(0.0) 
                mat_sum_cuda(drift_rate,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_rate[accumulator]   # Collecting all drift-rate except current
                mat_sum_cuda(drift_offset,temp)
                ddm_like_dr_inhibition_step += temp[0] - drift_offset[accumulator] # Collecting all drift-offset except current
                
                decision_step = decision_variable[accumulator] \
                                + diffusion_step \
#                                 - leak_step \
#                                 - lateral_dv_inhibition_step* \
#                                 - ddm_like_dr_inhibition_step*neural_ddm
                
                decision_variable[accumulator] = decision_step * (decision_step > 0)      # Implementing ReLU operation i.e., only non-negative value else zero
            
                if decision_variable[accumulator] > decision_bound[t]:
                    decision[tr] = accumulator
                    reaction_time[tr] = t + nondecision_time
                    break
            else:
                continue
            break

                
      
        
        

num_trials = 2#0000
stop_time = 100#00
num_choices = 2
coh = 100
coherence = np.ones((num_trials, stop_time), dtype=float32)*coh

starting_point = cp.array(np.zeros(num_choices), dtype=float32)             
drift_offset = cp.array(np.zeros(num_choices), dtype=float32)
drift_gain = 10e-6             # drift gain
drift_variability = 10e-3      # diffusion variability
nondecision_time = 100         # Non-decision time (msec)
decision_bound = 1
bound_rate = 0
bound_delay = 0
lateral_inhibition = 0
leak = 0
neural_ddm = 0

# Dynamic time-dependent variables
stimulus_np = get_unsigned_coherence_matrix(coherence)
decision_bound_cp = get_time_dependent_bound(decision_bound, bound_rate, bound_delay)
drift_variability_cp = get_time_dependent_variability(drift_variability)
stimulus_cp = cp.asarray(stimulus_np)

decision = cp.empty(stimulus_cp.shape[0])*cp.NaN
reaction_time = cp.empty(stimulus_cp.shape[0])*cp.NaN
blockdim = (256)
griddim = stimulus_cp.shape[0] // (blockdim) + 1
rng_states = random.create_xoroshiro128p_states(griddim * blockdim, seed=1)
cuda.synchronize()
%time independent_accumulating_kernel[griddim, blockdim](stimulus_cp, starting_point, drift_gain, drift_variability_cp, drift_offset, decision_bound_cp, nondecision_time, lateral_inhibition, leak, neural_ddm, decision, reaction_time, rng_states) 
cuda.synchronize()


Wall time: 345 ms


In [285]:
decision

array([2., 2.])